In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retail-products-classification-2023/how_to_download.txt


In [2]:
!pip install wget

  Preparing metadata (setup.py) ... - \ done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=32e746a5c36a9ec65eb45ab4f9f1154d766ff60bed2e7132ea576410190cb842
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import os
import pandas as pd
import random
import zipfile

In [4]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip

100% [....................................................] 28153268 / 28153268
Saved under test_data.zip


In [5]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1

100% [................................................] 1171218652 / 1171218652
Saved under products_leaflets_256.zip


In [6]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [7]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [8]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image


# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'
val_directory = 'products_leaflets_256/test'

# Bildgröße und Anzahl der Klassen
image_size = (256, 256)
num_classes = len(os.listdir(train_directory))

# Daten Augmentation
#data_augmentation = tf.keras.Sequential([
    #tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    #tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    #tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    #tf.keras.layers.RandomFlip(mode = 'horizontal')
#])

# Durchlaufe das Trainingsverzeichnis und lade die Bilder
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)

# Durchlaufe das Validierungsverzeichnis und lade die Bilder
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)






/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Found 33280 files belonging to 832 classes.
Found 8320 files belonging to 832 classes.


In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, DenseNet121, DenseNet201, DenseNet169
import tensorflow.keras.optimizers.legacy as legacy_optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
# Modellarchitektur anpassen
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
#base_model = MobileNetV2(include_top=False, input_shape=(256, 256, 3))
#base_model = DenseNet121(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
base_model = DenseNet169(include_top=False, pooling='max', weights = 'imagenet', input_shape=(256, 256, 3))
#base_model = DenseNet201(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
#base_model = ResNet50(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))

#base_model.trainable = false  # Basismodell trainierbar machen
# Data Augmentation erweitern
model = Sequential([
    #data_augmentation,
    base_model,
    #Dense(256, activation="relu"),
    #Dense(128, activation='relu'),
    #Dense(64, activation="relu"),
    Dense(num_classes, activation='softmax')
])
base_model.layers[0].trainable = False
# Kompilieren des Modells mit angepasster Lernrate


#sgd = legacy_optimizers.SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=1e-6)
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trainingsparameter anpassen
batch_size = 32
epochs = 16
#early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# Modelltraining mit erweitertem Data Augmentation
model.fit(train_ds,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=val_ds,
          callbacks=[reduce_lr])



51877672/51877672 [==============================] - 0s 0us/step
Epoch 1/16
1040/1040 [==============================] - 433s 309ms/step - loss: 2.0523 - accuracy: 0.7061 - val_loss: 0.8373 - val_accuracy: 0.8376 - lr: 1.0000e-04
Epoch 2/16
1040/1040 [==============================] - 319s 307ms/step - loss: 0.1137 - accuracy: 0.9778 - val_loss: 0.7338 - val_accuracy: 0.8653 - lr: 1.0000e-04
Epoch 3/16
1040/1040 [==============================] - 319s 306ms/step - loss: 0.0580 - accuracy: 0.9892 - val_loss: 0.7077 - val_accuracy: 0.8620 - lr: 1.0000e-04
Epoch 4/16
1040/1040 [==============================] - 319s 306ms/step - loss: 0.0797 - accuracy: 0.9844 - val_loss: 0.8844 - val_accuracy: 0.8345 - lr: 1.0000e-04
Epoch 5/16
1040/1040 [==============================] - 320s 307ms/step - loss: 0.0777 - accuracy: 0.9840 - val_loss: 0.8078 - val_accuracy: 0.8573 - lr: 1.0000e-04
Epoch 6/16
1040/1040 [==============================] - 319s 306ms/step - loss: 0.0139 - accuracy: 0.9971 - va

In [10]:
# Liste für Bildnamen und Vorhersagen
image_names = os.listdir('./test_data')
class_names = sorted(os.listdir(train_directory))
label_predictions = []
img_array = []
test_directory = 'test_data'


# Durchlaufe das Testverzeichnis und lade die Bilder
for image_name in image_names:
    url = "./test_data/" + image_name
    img = tf.keras.utils.load_img(url, target_size=image_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    label_predictions.append(class_names[np.argmax(score)])


# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

1/1 [==============================] - 0s 39ms/step


# Neuer Abschnitt